# 数据预处理

## 一、合并 train + test
去掉 Id、train 去掉 SalePrice

保证 one-hot 维度一致

避免 test 出现 train 没有的类别

In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
train = pd.read_csv("../data/raw/train.csv")
test = pd.read_csv("../data/raw/test.csv")

print(train.shape)
print(test.shape)

(1460, 81)
(1459, 80)


In [19]:
all_features = pd.concat((train.iloc[:, 1:-1], test.iloc[:, 1:]))

## 二、数值特征标准化

In [20]:
numberic_features = all_features.dtypes[all_features.dtypes != 'object'].index

all_features[numberic_features] = all_features[numberic_features].apply(
    lambda x:(x-x.mean())/x.std())
all_features[numberic_features] = all_features[numberic_features].fillna(0)

## 三、类别特征 one-hot
特征由79个增加到了330个

In [21]:
all_features = pd.get_dummies(all_features, dummy_na=True)
all_features.shape

(2919, 330)

## 四、拆回 train/test并转换为张量

In [24]:
all_features = all_features.astype(float)
n_train = train.shape[0]

train_features = torch.tensor(
    all_features[:n_train].values,
    dtype=torch.float32
)

test_features = torch.tensor(
    all_features[n_train:].values,
    dtype=torch.float32
)

train_labels = torch.tensor(
    train.SalePrice.values.reshape(-1,1),
    dtype=torch.float32
)

In [28]:
all_features.dtypes.value_counts()
train_features.shape,test_features.shape

(torch.Size([1460, 330]), torch.Size([1459, 330]))

## 五、对标签对log变换

In [29]:
train_labels = torch.log1p(train_labels)